In [1]:
import sys
import glob
import pickle
import numpy as np
import re
from datetime import datetime
from music21 import converter, note, chord, stream, instrument

from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Bidirectional, Dense, Dropout, LSTM, Activation
from keras.callbacks import ModelCheckpoint, TensorBoard, History, Callback
from keras.utils import np_utils

import functions as fxn

Using TensorFlow backend.


In [2]:
def full_execution(midi_files, output_tag, total_epochs, batch_size, sequence_length, temperature, offset_adj):
    # epoch_stops = 1
    # epoch_count = 0
    weight_file = None

    note_file = fxn.convert_midis_to_notes(midi_files, output_tag)

    epochs = total_epochs
    with open(note_file, 'rb') as filepath:
        notes = pickle.load(filepath)
    network_input, network_output, n_patterns, n_vocab, pitchnames = fxn.prepare_sequences(notes, sequence_length)
    network_input_r, network_output_r = fxn.reshape_for_training(network_input, network_output,sequence_length)

    # while epoch_count <= total_epochs:
    #     epochs = epoch_stops
    #
    model = fxn.create_network(network_input_r, n_vocab, weight_file)
    model, weight_file, history = fxn.train_model(model, network_input_r, network_output_r, epochs, batch_size, output_tag, sequence_length)
    normalized_input = fxn.reshape_for_creation(network_input, n_patterns, sequence_length, n_vocab)
    model = fxn.create_network(normalized_input, n_vocab, weight_file)
    prediction_output= fxn.generate_notes(model, network_input, pitchnames,sequence_length, notes_generated, n_vocab, temperature)
    output_notes = fxn.create_midi(prediction_output, output_tag, sequence_length, offset_adj)
        # epoch_count += epoch_stops
    return output_notes, history, weight_file

In [4]:
timestamp = re.sub(r'[-: ]','',str(datetime.now()).split('.')[0])[:-2]
total_epochs = 70
batch_size = 128 # 128 for local; 512 for AWS
sequence_length = 200 # the LSTM RNN will consider this many notes
notes_generated = 500
temperature = 1.0
offset_adj = 0.5
midi_files = '/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/*.mid'
output_name = midi_files.split('/')[-2]
output_tag = '/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/{}-{}-'.format(timestamp, output_name)

terminal_output = output_tag + 'terminal.log'
sys.stdout = fxn.Logger(terminal_output)
print("Terminal output being saved at {}".format(terminal_output))
full_execution(midi_files, output_tag, total_epochs, batch_size, sequence_length, temperature, offset_adj)
print("Run Complete. Terminal log saved at {}".format(terminal_output))

Terminal output being saved at /Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/201901052238-Country-terminal.log

**Loading Midi files**
/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/HungryEyes.mid
/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/WorkingmanBlues.mid
/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/BigCity.mid
/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/DaddyFrank.mid
/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/GeorgiaOnMyMind.mid
/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/AmIStandinginyourway.mid
/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/WildSideOfLife (1).mid
/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/Singmebackhome.mid
/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/Alwaysonmymind (1).mid
/Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/Country/IthinkIlljustgetdrunk.mid
/Users/anant.a.sharma/Desktop/Da

Network input and output created with (pre-transform) lengths 49110 and 49110
Lengths. N Vocab: 238 N Patterns: 49110 Pitchnames: 238

**Reshaping for training**
Reshaping network input to (notes - sequence length, sequence length) (49110, 200, 1)
Reshaping network output to (notes - sequence length, unique notes) (49110, 238)

**LSTM model initializing**
Input nodes: 512 Dropout: 0.5
Input shape (timesteps, data_dim): (200,1)
expected str, bytes or os.PathLike object, not NoneType
No weights loaded
Fitting Model. 
Network Input Shape: (49110, 200, 1) Network Output Shape: (49110, 238)
Epochs: 70 Batch Size: 128
Train on 39288 samples, validate on 9822 samples
Epoch 1/70
39288/39288 [==============================] - 2033s 52ms/step - loss: 4.8918 - val_loss: 4.3006

Epoch 00001: saving model to /Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/201901052238-Country-weight-01-4.8918-4.3006.hdf5
Epoch 2/70
39288/39288 [==============================] - 2059s 52ms/step - loss: 

39288/39288 [==============================] - 1903s 48ms/step - loss: 4.2911 - val_loss: 4.1722

Epoch 00031: saving model to /Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/201901052238-Country-weight-31-4.2911-4.1722.hdf5
Epoch 32/70
39288/39288 [==============================] - 1879s 48ms/step - loss: 4.2956 - val_loss: 4.2052

Epoch 00032: saving model to /Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/201901052238-Country-weight-32-4.2956-4.2052.hdf5
Epoch 33/70
39288/39288 [==============================] - 1913s 49ms/step - loss: 4.2831 - val_loss: 4.1805

Epoch 00033: saving model to /Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/201901052238-Country-weight-33-4.2831-4.1805.hdf5
Epoch 34/70
39288/39288 [==============================] - 1914s 49ms/step - loss: 4.2912 - val_loss: 4.1279

Epoch 00034: saving model to /Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/201901052238-Country-weight-34-4.2912-4.1279.hdf5
Epoch 35/70


Epoch 64/70
39288/39288 [==============================] - 1930s 49ms/step - loss: 4.2560 - val_loss: 4.2051

Epoch 00064: saving model to /Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/201901052238-Country-weight-64-4.2560-4.2051.hdf5
Epoch 65/70
39288/39288 [==============================] - 1921s 49ms/step - loss: 4.2486 - val_loss: 4.1713

Epoch 00065: saving model to /Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/201901052238-Country-weight-65-4.2486-4.1713.hdf5
Epoch 66/70
39288/39288 [==============================] - 1925s 49ms/step - loss: 4.2405 - val_loss: 4.1548

Epoch 00066: saving model to /Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/201901052238-Country-weight-66-4.2405-4.1548.hdf5
Epoch 67/70
39288/39288 [==============================] - 1919s 49ms/step - loss: 4.2446 - val_loss: 4.1940

Epoch 00067: saving model to /Users/anant.a.sharma/Desktop/Darya/AI_for_Music/DATA/output/201901052238-Country-weight-67-4.2446-4.1940.hdf5
